# Naive Bayes Third Round
26/11/2016  
ironbar

I'm going to use the new splitted dataset to train a naive bayes model. I will be using a class for the dataset, and this will help me for later using a unified class for the model. 

## Load the dataset

In [1]:
#Imports
import numpy as np
from sklearn.naive_bayes import BernoulliNB
import time
import gzip
import warnings
warnings.filterwarnings("ignore")

from dataset import SantanderDataset
from average_precision import mapk
from genetic_search import genetic_search

/Users/liouscott/anaconda/lib/python3.6/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


In [2]:
dataset_root = '/Users/liouscott/Documents/scott/kaggle/Genetic Algorithms/'
dataset = SantanderDataset(dataset_root)

It took 3 seconds to load the dataset
It took 6 seconds to load the dataset
It took 7 seconds to load the dataset
It took 2 seconds to load the dataset
1375686 1375686
561234 561234
{'ind_empleado': {'B': 0, 'F': 1, 'A': 2, 'N': 3, 'S': 4}, 'pais_residencia': {'ES': 0, 'US': 1, 'CH': 2, 'MX': 3, 'GB': 4, 'BE': 5, 'NG': 6, 'RU': 7, 'MA': 8, 'DK': 9, 'AE': 10, 'CN': 11, 'CA': 12, 'AR': 13, 'SG': 14, 'NL': 15, 'IT': 16, 'FR': 17, 'DE': 18, 'PL': 19, 'GQ': 20, 'CO': 21, 'PE': 22, 'OM': 23, 'CL': 24, 'AT': 25, 'CM': 26, 'SA': 27, 'CI': 28, 'IN': 29, 'PA': 30, 'ZA': 31, 'BR': 32, 'JP': 33, 'AU': 34, 'SE': 35, 'KE': 36, 'PT': 37, 'VE': 38, 'BO': 39, 'CR': 40, 'GR': 41, 'BY': 42, 'HK': 43, 'TR': 44, 'AD': 45, 'VN': 46, 'PR': 47, 'IE': 48, 'ET': 49, 'AO': 50, 'FI': 51, 'LU': 52, 'GA': 53, 'DO': 54, 'NO': 55, 'CZ': 56, 'QA': 57, 'CG': 58, 'NZ': 59, 'BM': 60, 'HN': 61, 'MT': 62, 'IL': 63, 'RO': 64, 'RS': 65, 'JM': 66, 'CU': 67, 'EC': 68, 'KR': 69, 'GH': 70, 'KH': 71, 'MM': 72, 'UY': 73, 'NI': 74,

         fecha_dato      ncodpers  ind_ahor_fin_ult1  ind_aval_fin_ult1  \
count  1.375686e+06  1.375686e+06       1.375686e+06       1.375686e+06   
mean   1.333430e+01  8.553411e+05       1.017674e-04       3.561859e-05   
std    4.624937e+00  4.601497e+05       1.008747e-02       5.968027e-03   
min    0.000000e+00  1.588900e+04       0.000000e+00       0.000000e+00   
25%    1.200000e+01  4.408522e+05       0.000000e+00       0.000000e+00   
50%    1.600000e+01  9.397025e+05       0.000000e+00       0.000000e+00   
75%    1.600000e+01  1.258222e+06       0.000000e+00       0.000000e+00   
max    1.600000e+01  1.553689e+06       1.000000e+00       1.000000e+00   

       ind_cco_fin_ult1  ind_cder_fin_ult1  ind_cno_fin_ult1  \
count      1.375686e+06       1.375686e+06      1.375686e+06   
mean       5.752490e-01       5.168331e-04      1.576108e-01   
std        4.943053e-01       2.272810e-02      3.643758e-01   
min        0.000000e+00       0.000000e+00      0.000000e+00   
25% 

In [4]:
dataset

I have been testing the class and seems to be working fine. 
When loaded the dataset is using only 500MB of RAM.

## Testing with Naive Bayes

In [3]:
def train_bnb_model(msg):
    """
    Trains a model using the given parameters
    
    month: int or list with the number of the month we want
        the data to be taken of
    input_columns: a list with the name of the columns we are going to use
        in the task
    use_product: bool, if true adds the product columns of the month before
    use_change: bool, if true adds the change columns of the month before
    """
    msg_copy = msg.copy()
    msg_copy['train'] = True
    if not 'month' in msg_copy.keys():
        msg_copy['month'] = msg_copy['train_month']
    #Get the data for training
    ret = dataset.get_data(msg_copy)
    input_data, output_data = ret[0:2]
    #print (type((ret[1])))
    #Fit the model
    bnb = BernoulliNB(alpha=1e-2)
    print (input_data,output_data)
    bnb.partial_fit(input_data, output_data, classes = range(24))
    return bnb

In [11]:
def create_prediction(bnb, msg):
    """
    Makes a prediction using the given model and parameters
    
    month: int or list with the number of the month we want
        the data to be taken of
    input_columns: a list with the name of the columns we are going to use
        in the task
    use_product: bool, if true adds the product columns of the month before
    use_change: bool, if true adds the change columns of the month before
    """
    msg_copy = msg.copy()
    msg_copy['train'] = False
    if not 'month' in msg_copy.keys():
        msg_copy['month'] = msg_copy['eval_month']
    #Get the data for making a prediction
    ret = dataset.get_data(msg_copy)
    input_data, output_data, previous_products = ret
    #Get the prediction
    rank = bnb.predict_proba(input_data)
    filtered_rank = np.equal(previous_products, 0) * rank
    print (filtered_rank)
    print (rank)
    print (previous_products)
    print (np.equal(previous_products, 0))
    predictions = np.argsort(filtered_rank, axis=1)
    predictions = predictions[:,::-1][:,0:7]
    return predictions, output_data

In [5]:
def naive_bayes_workflow(msg):
    """
    Implements all the steps of training and evaluating a naive bayes classifier
    Returns the score and the trained model
    
    train_month: int or list with the number of the month we want
        the data to be taken of for training 
    eval_month: int or list with the number of the month we want
        the data to be taken of for testing
    input_columns: a list with the name of the columns we are going to use
        in the task
    use_product: bool, if true adds the product columns of the month before
    use_change: bool, if true adds the change columns of the month before
    """
    if type(msg['eval_month']) is not list:
        msg['eval_month'] = [msg['eval_month']]
    #Train the model
    bnb = train_bnb_model(msg)
    scores = []
    for month in msg['eval_month']:
        msg_copy = msg.copy()
        msg_copy['month'] = month
        #Create prediction
        predictions, output_data = create_prediction(bnb, msg_copy)
        #Get the score
        score = mapk(output_data, predictions)
        scores.append(score)
    
    return scores, bnb

In [12]:
#Try training
start_time = time.time()
msg = {'train_month': 5,
       'eval_month': 5,
      'input_columns': [],
      'use_product': True,
      'use_change': False}
print (naive_bayes_workflow(msg)[0])
print (time.time()-start_time)

{'train_month': 5, 'eval_month': [5], 'input_columns': [], 'use_product': True, 'use_change': False, 'train': True, 'month': 5}
[[0 0 0 ..., 0 0 1]
 [0 0 1 ..., 0 0 1]
 [0 0 0 ..., 0 0 1]
 ..., 
 [0 0 1 ..., 0 0 0]
 [0 0 1 ..., 0 0 0]
 [0 0 1 ..., 0 0 0]] [ 2 22 22 ..., 23 21 22]
{'train_month': 5, 'eval_month': [5], 'input_columns': [], 'use_product': True, 'use_change': False, 'month': 5, 'train': False}
[[ 0.          0.          0.02615482 ...,  0.0864693   0.86867199  0.        ]
 [ 0.          0.          0.         ...,  0.13490843  0.84545853  0.        ]
 [ 0.          0.          0.02622663 ...,  0.16744893  0.71335329  0.        ]
 ..., 
 [ 0.          0.          0.         ...,  0.02220982  0.01133845
   0.62068561]
 [ 0.          0.          0.         ...,  0.02220982  0.01133845
   0.62068561]
 [ 0.          0.          0.         ...,  0.02220982  0.01133845
   0.62068561]]
[[  0.00000000e+00   0.00000000e+00   2.61548203e-02 ...,   8.64693033e-02
    8.68671991e-01   

In [9]:
#Try training
start_time = time.time()
msg = {'train_month': 5,
       'eval_month': 5,
      'input_columns': dataset.categorical_columns,
      'use_product': True,
      'use_change': False}
print (naive_bayes_workflow(msg)[0])
print (time.time()-start_time)

{'train_month': 5, 'eval_month': [5], 'input_columns': ['ind_empleado', 'pais_residencia', 'sexo', 'age', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'indfall', 'nomprov', 'ind_actividad_cliente', 'renta', 'segmento', 'month'], 'use_product': True, 'use_change': False, 'train': True, 'month': 5}
(41738, 435) (41738, 24)
<class 'numpy.ndarray'>
[[0 1 0 ..., 0 0 1]
 [0 0 1 ..., 0 0 1]
 [0 0 0 ..., 0 0 1]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]] [ 2 22 22 ..., 23 21 22]
{'train_month': 5, 'eval_month': [5], 'input_columns': ['ind_empleado', 'pais_residencia', 'sexo', 'age', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'indfall', 'nomprov', 'ind_actividad_cliente', 'renta', 'segmento', 'month'], 'use_product': True, 'use_change': False, 'month': 5, 'train': False}
(33312, 435) (33312, 24)
[0.70271425855404612]
1.3501009941101074


In [10]:
#Try training
start_time = time.time()
msg = {'train_month': list(range(1,17)),
       'eval_month': 5,
      'input_columns': dataset.categorical_columns,
      'use_product': True,
      'use_change': False}
print (naive_bayes_workflow(msg)[0])
print (time.time()-start_time)

{'train_month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 'eval_month': [5], 'input_columns': ['ind_empleado', 'pais_residencia', 'sexo', 'age', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'indfall', 'nomprov', 'ind_actividad_cliente', 'renta', 'segmento', 'month'], 'use_product': True, 'use_change': False, 'train': True, 'month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]}
{'train_month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 'eval_month': [5], 'input_columns': ['ind_empleado', 'pais_residencia', 'sexo', 'age', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'indfall', 'nomprov', 'ind_actividad_cliente', 'renta', 'segmento', 'month'], 'use_product': True, 'use_change': False, 'train': True, 'month': 1}
(31204, 435) (31204, 24)
{'train_month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 'eval_month': [5], 'inpu

In [11]:
#Try training
start_time = time.time()
msg = {'train_month': 5,
       'eval_month': [5,16],
      'input_columns': [],
      'use_product': True,
      'use_change': False}
print (naive_bayes_workflow(msg)[0])
print (time.time()-start_time)

{'train_month': 5, 'eval_month': [5, 16], 'input_columns': [], 'use_product': True, 'use_change': False, 'train': True, 'month': 5}
<class 'numpy.ndarray'>
[[0 0 0 ..., 0 0 1]
 [0 0 1 ..., 0 0 1]
 [0 0 0 ..., 0 0 1]
 ..., 
 [0 0 1 ..., 0 0 0]
 [0 0 1 ..., 0 0 0]
 [0 0 1 ..., 0 0 0]] [ 2 22 22 ..., 23 21 22]
{'train_month': 5, 'eval_month': [5, 16], 'input_columns': [], 'use_product': True, 'use_change': False, 'month': 5, 'train': False}
{'train_month': 5, 'eval_month': [5, 16], 'input_columns': [], 'use_product': True, 'use_change': False, 'month': 16, 'train': False}
[0.70998466756819612, 0.69814177663890675]
0.4544360637664795


In [12]:
#Try training
start_time = time.time()
msg = {'train_month': [ 1,  2,  3,  6,  9, 12, 14, 15],
       'eval_month': [5,16],
      'input_columns': ['pais_residencia', 'age', 'antiguedad', 'tiprel_1mes',
                        'indresi', 'indext', 'canal_entrada', 'nomprov', 'renta',
                        'segmento', 'month'],
      'use_product': True,
      'use_change': False}
print (naive_bayes_workflow(msg)[0])
print (time.time()-start_time)

{'train_month': [1, 2, 3, 6, 9, 12, 14, 15], 'eval_month': [5, 16], 'input_columns': ['pais_residencia', 'age', 'antiguedad', 'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'nomprov', 'renta', 'segmento', 'month'], 'use_product': True, 'use_change': False, 'train': True, 'month': [1, 2, 3, 6, 9, 12, 14, 15]}
{'train_month': [1, 2, 3, 6, 9, 12, 14, 15], 'eval_month': [5, 16], 'input_columns': ['pais_residencia', 'age', 'antiguedad', 'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'nomprov', 'renta', 'segmento', 'month'], 'use_product': True, 'use_change': False, 'train': True, 'month': 1}
(31204, 417) (31204, 24)
{'train_month': [1, 2, 3, 6, 9, 12, 14, 15], 'eval_month': [5, 16], 'input_columns': ['pais_residencia', 'age', 'antiguedad', 'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'nomprov', 'renta', 'segmento', 'month'], 'use_product': True, 'use_change': False, 'train': True, 'month': 2}
(31803, 417) (31803, 24)
{'train_month': [1, 2, 3, 6, 9, 12, 14, 15], 'eval_mon

This is very good, it seems to be working correctly and it's much faster than the other implementations, in the worst case I will be running 4 experiments per minute. And in the best case it will run 240 experiments per minute.

In the previos implementation the average time was 2 experiments per minute.
So we get an speedup of 2 to 120.

## Genetic algorithm search

I'm going to launch a new search to see if I can get the same results or better.

In [13]:
def get_genomic_score(test_month, filename, genome, verbose=False):
    """
    Receives only test month and the genome
    Returns the score and saves the configuration and results in a file
    
    If the genome size is 34 then use_product is set to True
    If the genome size is 35 then all the parameters are in the search
    len(categorical_columns) = 18
    So we need a genome of 18+2+1 = 21
    """
    if verbose: print (genome)
    #Decide which train months to use, we can train from month 1 to 16
    if np.sum(genome[0:15]) > 0:
        used_months = np.array(list(range(1,5))+list(range(6,17)))[np.array(genome[0:15]) == 1]
        train_month = used_months
    else:
        #Select a random month, excluding 5
        used_months = np.random.randint(1,16,1)[0]
        if used_months >= 5: used_months += 1
        train_month = [used_months]
    if verbose: print ('train_month', train_month)
    #Decide wich category input columns to use
    categorical_columns = dataset.categorical_columns
    #print (categorical_columns)
    used_index = np.arange(len(categorical_columns))[
        np.array(genome[15:33]) == 1]
    input_columns = [categorical_columns[i] for i in used_index]
    if verbose: print ('input_columns', input_columns)
    #Decide on using change columns and product as input
    use_change = genome[33] == 1
    #This allows to use a shorter genome to fix some properties
    if len(genome) >= 35: 
        use_product = genome[34] == 1
    else:
        use_product = True
    #Build message for training
    print (train_month)
    msg ={'train_month':list(train_month),
          'eval_month':test_month,
          'input_columns':input_columns,
          'use_product':use_product,
          'use_change':use_change,
        
    }
    if verbose: print (msg)
    ret = naive_bayes_workflow(msg)
    #Print and save to file 
    text = '\t'.join([str(a) for a in ret[0]]) + '\t'
    text += '%s\t%s\t' % ( use_change, use_product)
    if verbose: print (text)
    text += "','".join(input_columns)
    text += "\t" + ",".join([str(a) for a in train_month])
    text += '\n'
    with open(dataset_root+'logs/%s.log' % filename, 'a') as f:
        f.write(text)
        
    return ret[0]

In [14]:
#Define evaluation function
def eval_function_1(individual):
    """
    Tries to optimize just the training score
    """
    print (individual)
    ret = get_genomic_score([5,16],'genetic_search_6',individual,verbose=False)
    return ret[0:1]

In [15]:
print ('Starting genetic search')
genetic_search(eval_function_1, 35, 20)

Starting genetic search
<function eval_function_1 at 0x118662a60>
[1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0]
[ 1  2 10 11 13 14 16]
{'train_month': [1, 2, 10, 11, 13, 14, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'pais_residencia', 'age', 'indrel_1mes', 'indext', 'canal_entrada', 'ind_actividad_cliente', 'renta', 'segmento'], 'use_product': False, 'use_change': False, 'train': True, 'month': [1, 2, 10, 11, 13, 14, 16]}
{'train_month': [1, 2, 10, 11, 13, 14, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'pais_residencia', 'age', 'indrel_1mes', 'indext', 'canal_entrada', 'ind_actividad_cliente', 'renta', 'segmento'], 'use_product': False, 'use_change': False, 'train': True, 'month': 1}
{'train_month': [1, 2, 10, 11, 13, 14, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'pais_residencia', 'age', 'indrel_1mes', 'indext', 'canal_entrada', 'ind_actividad_cliente', 'renta', 'segm

(31204, 137) (31204, 70)
{'train_month': [1, 6, 8, 9, 10, 12, 13, 14, 15, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'antiguedad', 'indrel_1mes', 'nomprov', 'ind_actividad_cliente', 'renta'], 'use_product': True, 'use_change': True, 'train': True, 'month': 6}
(33362, 113) (33362, 24)
(33362, 137) (33362, 70)
{'train_month': [1, 6, 8, 9, 10, 12, 13, 14, 15, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'antiguedad', 'indrel_1mes', 'nomprov', 'ind_actividad_cliente', 'renta'], 'use_product': True, 'use_change': True, 'train': True, 'month': 8}
(35696, 113) (35696, 24)
(35696, 137) (35696, 70)
{'train_month': [1, 6, 8, 9, 10, 12, 13, 14, 15, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'antiguedad', 'indrel_1mes', 'nomprov', 'ind_actividad_cliente', 'renta'], 'use_product': True, 'use_change': True, 'train': True, 'month': 9}
(39803, 113) (39803, 24)
(39803, 137) (39803, 70)
{'train_month': [1, 6, 8, 9, 10, 12, 13, 14, 15, 16], 'eval_mon

(35843, 252) (35843, 24)
<class 'numpy.ndarray'>
[[1 0 1 ..., 1 1 1]
 [1 0 1 ..., 0 0 1]
 [1 0 1 ..., 0 0 1]
 ..., 
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]] [ 4 13  2 ...,  2  2  2]
{'train_month': [6, 12, 16], 'eval_month': [5, 16], 'input_columns': ['indresi', 'canal_entrada', 'indfall', 'nomprov', 'renta', 'segmento'], 'use_product': True, 'use_change': False, 'month': 5, 'train': False}
(33312, 252) (33312, 24)
{'train_month': [6, 12, 16], 'eval_month': [5, 16], 'input_columns': ['indresi', 'canal_entrada', 'indfall', 'nomprov', 'renta', 'segmento'], 'use_product': True, 'use_change': False, 'month': 16, 'train': False}
(27875, 252) (27875, 24)
[0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[ 4  6 11 13 14 16]
{'train_month': [4, 6, 11, 13, 14, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'pais_residencia', 'antiguedad', 'indrel', 'indresi', 'canal_entrada'], 'use_product': False, 'use_c

(35215, 224) (35215, 24)
{'train_month': [1, 2, 3, 8, 9, 10, 11, 12, 13, 15], 'eval_month': [5, 16], 'input_columns': ['sexo', 'age', 'indrel', 'indext', 'canal_entrada', 'renta', 'segmento', 'month'], 'use_product': True, 'use_change': False, 'train': True, 'month': 11}
(42353, 224) (42353, 24)
{'train_month': [1, 2, 3, 8, 9, 10, 11, 12, 13, 15], 'eval_month': [5, 16], 'input_columns': ['sexo', 'age', 'indrel', 'indext', 'canal_entrada', 'renta', 'segmento', 'month'], 'use_product': True, 'use_change': False, 'train': True, 'month': 12}
(30956, 224) (30956, 24)
{'train_month': [1, 2, 3, 8, 9, 10, 11, 12, 13, 15], 'eval_month': [5, 16], 'input_columns': ['sexo', 'age', 'indrel', 'indext', 'canal_entrada', 'renta', 'segmento', 'month'], 'use_product': True, 'use_change': False, 'train': True, 'month': 13}
(49024, 224) (49024, 24)
{'train_month': [1, 2, 3, 8, 9, 10, 11, 12, 13, 15], 'eval_month': [5, 16], 'input_columns': ['sexo', 'age', 'indrel', 'indext', 'canal_entrada', 'renta', 'seg

(30339, 237) (30339, 70)
{'train_month': [3, 6, 7, 8, 12, 13, 14, 15, 16], 'eval_month': [5, 16], 'input_columns': ['pais_residencia', 'sexo', 'age', 'antiguedad', 'indrel', 'tiprel_1mes', 'nomprov', 'segmento'], 'use_product': True, 'use_change': True, 'train': True, 'month': 6}
(33362, 213) (33362, 24)
(33362, 237) (33362, 70)
{'train_month': [3, 6, 7, 8, 12, 13, 14, 15, 16], 'eval_month': [5, 16], 'input_columns': ['pais_residencia', 'sexo', 'age', 'antiguedad', 'indrel', 'tiprel_1mes', 'nomprov', 'segmento'], 'use_product': True, 'use_change': True, 'train': True, 'month': 7}
(29490, 213) (29490, 24)
(29490, 237) (29490, 70)
{'train_month': [3, 6, 7, 8, 12, 13, 14, 15, 16], 'eval_month': [5, 16], 'input_columns': ['pais_residencia', 'sexo', 'age', 'antiguedad', 'indrel', 'tiprel_1mes', 'nomprov', 'segmento'], 'use_product': True, 'use_change': True, 'train': True, 'month': 8}
(35696, 213) (35696, 24)
(35696, 237) (35696, 70)
{'train_month': [3, 6, 7, 8, 12, 13, 14, 15, 16], 'eval_m

...

(33312, 209) (33312, 70)
{'train_month': [2, 4, 6, 8, 10, 11, 13, 14, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'pais_residencia', 'indrel_1mes', 'indresi', 'indext', 'nomprov', 'ind_actividad_cliente'], 'use_product': True, 'use_change': True, 'month': 16, 'train': False}
(27875, 185) (27875, 24)
(27875, 209) (27875, 70)
[0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1]
[ 6  8 10 11 13 14 16]
{'train_month': [6, 8, 10, 11, 13, 14, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'pais_residencia', 'indrel_1mes', 'indext', 'canal_entrada'], 'use_product': True, 'use_change': True, 'train': True, 'month': [6, 8, 10, 11, 13, 14, 16]}
{'train_month': [6, 8, 10, 11, 13, 14, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'pais_residencia', 'indrel_1mes', 'indext', 'canal_entrada'], 'use_product': True, 'use_change': True, 'train': True, 'month': 6}
(33362, 291) (33362, 24)
(33362, 315) (3

(42353, 315) (42353, 70)
{'train_month': [6, 7, 11, 13, 14, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'pais_residencia', 'indrel_1mes', 'indext', 'canal_entrada'], 'use_product': True, 'use_change': True, 'train': True, 'month': 13}
(49024, 291) (49024, 24)
(49024, 315) (49024, 70)
{'train_month': [6, 7, 11, 13, 14, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'pais_residencia', 'indrel_1mes', 'indext', 'canal_entrada'], 'use_product': True, 'use_change': True, 'train': True, 'month': 14}
(35203, 291) (35203, 24)
(35203, 315) (35203, 70)
{'train_month': [6, 7, 11, 13, 14, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'pais_residencia', 'indrel_1mes', 'indext', 'canal_entrada'], 'use_product': True, 'use_change': True, 'train': True, 'month': 16}
(35843, 291) (35843, 24)
(35843, 315) (35843, 70)
<class 'numpy.ndarray'>
[[0 0 1 ..., 1 0 0]
 [0 1 0 ..., 1 0 0]
 [0 1 0 ..., 1 0 0]
 ..., 
 [0 0 0 ..., 1 0 0]
 [0 0 0 ..., 1 0 0]
 [0 0 0 ..

(26243, 209) (26243, 70)
{'train_month': [2, 4, 6, 8, 10, 11, 13, 14, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'pais_residencia', 'indrel_1mes', 'indresi', 'indext', 'nomprov', 'ind_actividad_cliente'], 'use_product': True, 'use_change': True, 'train': True, 'month': 6}
(33362, 185) (33362, 24)
(33362, 209) (33362, 70)
{'train_month': [2, 4, 6, 8, 10, 11, 13, 14, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'pais_residencia', 'indrel_1mes', 'indresi', 'indext', 'nomprov', 'ind_actividad_cliente'], 'use_product': True, 'use_change': True, 'train': True, 'month': 8}
(35696, 185) (35696, 24)
(35696, 209) (35696, 70)
{'train_month': [2, 4, 6, 8, 10, 11, 13, 14, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'pais_residencia', 'indrel_1mes', 'indresi', 'indext', 'nomprov', 'ind_actividad_cliente'], 'use_product': True, 'use_change': True, 'train': True, 'month': 10}
(35215, 185) (35215, 24)
(35215, 209) (35215, 70)
{'train_month': [2, 4, 

(33362, 376) (33362, 70)
{'train_month': [2, 4, 6, 8, 9, 10, 11, 13, 14, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'pais_residencia', 'sexo', 'indrel', 'indrel_1mes', 'indresi', 'indext', 'canal_entrada', 'nomprov', 'ind_actividad_cliente'], 'use_product': True, 'use_change': True, 'train': True, 'month': 8}
(35696, 352) (35696, 24)
(35696, 376) (35696, 70)
{'train_month': [2, 4, 6, 8, 9, 10, 11, 13, 14, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'pais_residencia', 'sexo', 'indrel', 'indrel_1mes', 'indresi', 'indext', 'canal_entrada', 'nomprov', 'ind_actividad_cliente'], 'use_product': True, 'use_change': True, 'train': True, 'month': 9}
(39803, 352) (39803, 24)
(39803, 376) (39803, 70)
{'train_month': [2, 4, 6, 8, 9, 10, 11, 13, 14, 16], 'eval_month': [5, 16], 'input_columns': ['ind_empleado', 'pais_residencia', 'sexo', 'indrel', 'indrel_1mes', 'indresi', 'indext', 'canal_entrada', 'nomprov', 'ind_actividad_cliente'], 'use_product': True, 'use_ch

([[0,
   1,
   0,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   1],
  [0,
   1,
   0,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   1],
  [0,
   0,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   1],
  [0,
   0,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   1],
  [0,
   0,
   0,
   0,
   1,
   0,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   1,


0.774615 is 0.04 points better than my previous best score, so I have to make a submission of this

### Optimizing for month 5 and 16
That search took only a few minutes. So I'm thinking of launching a search with population 10x size_vector which was the recommended parameter.  
Moreover I think that a good combination will be to maximize the sum of month 5 and 16, because we have discovered that both are important. If doing so I can't use the month 16 for training

In [19]:
def get_genomic_score(test_month, filename, genome, verbose=False):
    """
    Receives only test month and the genome
    Returns the score and saves the configuration and results in a file
    It's the same function as above but without training with month 16
    
    If the genome size is 33 then use_product is set to True
    If the genome size is 34 then all the parameters are in the search
    len(categorical_columns) = 18
    So we need a genome of 18+2+1 = 21
    """
    if verbose: print (genome)
    #Decide which train months to use, we can train from month 1 to 15
    if np.sum(genome[0:14]) > 0:
        used_months = np.array(list(range(1,5))+list(range(6,16)))[np.array(genome[0:14]) == 1]
        train_month = used_months
    else:
        #Select a random month, excluding 5 and 16
        used_months = np.random.randint(1,15,1)[0]
        if used_months >= 5: used_months += 1
        train_month = [used_months]
    if verbose: print ('train_month', train_month)
    #Decide wich category input columns to use
    categorical_columns = dataset.categorical_columns
    used_index = np.arange(len(categorical_columns))[
        np.array(genome[14:32]) == 1]
    input_columns = [categorical_columns[i] for i in used_index]
    if verbose: print ('input_columns', input_columns)
    #Decide on using change columns and product as input
    use_change = genome[32] == 1
    #This allows to use a shorter genome to fix some properties
    if len(genome) >= 34: 
        use_product = genome[33] == 1
    else:
        use_product = True
    #Build message for training 
    msg ={'train_month':list(train_month),
          'eval_month':test_month,
          'input_columns':input_columns,
          'use_product':use_product,
          'use_change':use_change,
        
    }
    if verbose: print (msg)
    ret = naive_bayes_workflow(msg)
    #Print and save to file 
    text = '\t'.join([str(a) for a in ret[0]]) + '\t'
    text += '%s\t%s\t' % ( use_change, use_product)
    if verbose: print (text)
    text += "','".join(input_columns)
    text += "\t" + ",".join([str(a) for a in train_month])
    text += '\n'
    with open(dataset_root+'logs/%s.log' % filename, 'a') as f:
        f.write(text)
        
    return ret[0]

In [16]:
#Define evaluation function
def eval_function_2(individual):
    """
    Tries to optimize just the training score
    """
    ret = get_genomic_score([5,16],'genetic_search_8',individual,verbose=False)
    return [np.sum(ret)/2]

In [20]:
print ('Starting genetic search')
genetic_search(eval_function_2, 34, 340)

Starting genetic search
gen	nevals	avg     	min     	max    
0  	340   	0.602454	0.162903	0.78162
1  	194   	0.700568	0.413165	0.78382
2  	208   	0.739201	0.450061	0.785003
3  	205   	0.756063	0.519034	0.79029 
4  	190   	0.765137	0.571924	0.787955
5  	236   	0.768146	0.435766	0.789901
6  	189   	0.770982	0.168229	0.789901
7  	205   	0.777759	0.597062	0.790683
8  	186   	0.777748	0.599718	0.791059
9  	210   	0.77853 	0.492293	0.79118 
10 	212   	0.780826	0.57914 	0.794627
Best individual is: [0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1]
with fitness: (0.79462683446643512,)
The search took -216 minutes.


([[0,
   0,
   1,
   1,
   1,
   0,
   0,
   1,
   1,
   1,
   0,
   1,
   0,
   0,
   0,
   1,
   1,
   1,
   0,
   0,
   1,
   0,
   0,
   1,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   1],
  [0,
   1,
   1,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   1,
   1,
   0,
   1,
   0,
   0,
   1,
   0,
   0,
   1,
   1],
  [0,
   0,
   0,
   0,
   1,
   0,
   0,
   1,
   1,
   1,
   1,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   0,
   1,
   1],
  [0,
   0,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   1,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   1],
  [0,
   0,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   1,
   0,
   1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,


### Optimizing and training only with month 5
I want to know what the maximun score I can get using only month 5 as input. I don't think this will get a good score at the LB, but it will give me a measure of the overfitting capacity of Naive Bayes

In [21]:
def get_genomic_score(test_month, filename, genome, verbose=False):
    """
    Receives only test month and the genome
    Returns the score and saves the configuration and results in a file
    In this version only trains with month 5
    
    If the genome size is 33 then use_product is set to True
    If the genome size is 34 then all the parameters are in the search
    len(categorical_columns) = 18
    So we need a genome of 18+2+1 = 21
    """
    if verbose: print (genome)
    train_month = [5]
    if verbose: print ('train_month', train_month)
    #Decide wich category input columns to use
    categorical_columns = dataset.categorical_columns
    used_index = np.arange(len(categorical_columns))[
        np.array(genome[0:18]) == 1]
    input_columns = [categorical_columns[i] for i in used_index]
    if verbose: print ('input_columns', input_columns)
    #Decide on using change columns and product as input
    use_change = genome[18] == 1
    #This allows to use a shorter genome to fix some properties
    if len(genome) >= 20: 
        use_product = genome[19] == 1
    else:
        use_product = True
    #Build message for training 
    msg ={'train_month':list(train_month),
          'eval_month':test_month,
          'input_columns':input_columns,
          'use_product':use_product,
          'use_change':use_change,
        
    }
    if verbose: print (msg)
    ret = naive_bayes_workflow(msg)
    #Print and save to file 
    text = '\t'.join([str(a) for a in ret[0]]) + '\t'
    text += '%s\t%s\t' % ( use_change, use_product)
    if verbose: print (text)
    text += "','".join(input_columns)
    text += "\t" + ",".join([str(a) for a in train_month])
    text += '\n'
    with open(dataset_root+'logs/%s.log' % filename, 'a') as f:
        f.write(text)
        
    return ret[0]

In [22]:
#Define evaluation function
def eval_function_3(individual):
    """
    Tries to optimize just the training score
    """
    ret = get_genomic_score([5,16],'genetic_search_9',individual,verbose=False)
    return ret[0:1]

In [23]:
print ('Starting genetic search')
genetic_search(eval_function_3, 20, 200) 

Starting genetic search
gen	nevals	avg     	min     	max     
0  	200   	0.720026	0.580795	0.808975
1  	117   	0.776169	0.604512	0.80995 
2  	124   	0.79637 	0.605697	0.809642
3  	121   	0.804295	0.711868	0.810434
4  	136   	0.803501	0.617131	0.811627
5  	130   	0.806226	0.713412	0.811627
6  	125   	0.805943	0.717044	0.811627
7  	107   	0.805096	0.629467	0.811654
8  	133   	0.806553	0.713011	0.811738
9  	131   	0.806096	0.618557	0.811713
10 	107   	0.808477	0.715517	0.811713
Best individual is: [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1]
with fitness: (0.81173807427229605,)
The search took -44 minutes.


([[1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1],
  [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1],
  [0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1],
  [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1],
  [0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1],
  [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1],
  [1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1],
  [1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1],
  [0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1],
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1],
  [1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1],
  [1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1],
  [1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1],
  [1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1],
  [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1],
  [0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1

### Optimizing with all the months
The idea is to leave the option of using all the months for training. And optimize the sum of the month 5 and 16.  
I have seen that optimizing wiht month 5 only for month 5 was not a good idea. But I think optimizing for both months at the same time will get a greater score even if I use those months for also for training

In [24]:
def get_genomic_score(test_month, filename, genome, verbose=False):
    """
    Receives only test month and the genome
    Returns the score and saves the configuration and results in a file
    It's the same function as above but without training with month 16
    
    If the genome size is 35 then use_product is set to True
    If the genome size is 36 then all the parameters are in the search
    len(categorical_columns) = 18
    So we need a genome of 18+2+1 = 21
    """
    if verbose: print (genome)
    #Decide which train months to use, from 1 to 16
    if np.sum(genome[0:16]) > 0:
        used_months = np.array(range(1,17))[np.array(genome[0:16]) == 1]
        train_month = used_months
    else:
        #Select a random month
        used_months = np.random.randint(1,17,1)[0]
        train_month = [used_months]
    if verbose: print ('train_month', train_month)
    #Decide wich category input columns to use
    categorical_columns = dataset.categorical_columns
    used_index = np.arange(len(categorical_columns))[
        np.array(genome[16:34]) == 1]
    input_columns = [categorical_columns[i] for i in used_index]
    if verbose: print ('input_columns', input_columns)
    #Decide on using change columns and product as input
    use_change = genome[34] == 1
    #This allows to use a shorter genome to fix some properties
    if len(genome) >= 36: 
        use_product = genome[35] == 1
    else:
        use_product = True
    #Build message for training 
    msg ={'train_month':list(train_month),
          'eval_month':test_month,
          'input_columns':input_columns,
          'use_product':use_product,
          'use_change':use_change,
        
    }
    if verbose: print (msg)
    ret = naive_bayes_workflow(msg)
    #Print and save to file 
    text = '\t'.join([str(a) for a in ret[0]]) + '\t'
    text += '%s\t%s\t' % ( use_change, use_product)
    if verbose: print (text)
    text += "','".join(input_columns)
    text += "\t" + ",".join([str(a) for a in train_month])
    text += '\n'
    with open(dataset_root+'logs/%s.log' % filename, 'a') as f:
        f.write(text)
        
    return ret[0]

In [25]:
#Define evaluation function
def eval_function_4(individual):
    """
    Tries to optimize just the training score
    """
    ret = get_genomic_score([5,16],'genetic_search_10',individual,verbose=False)
    return [np.sum(ret)/2]

In [27]:
print ('Starting genetic search')
start_time = time.time()
genetic_search(eval_function_4, 36, 600)
print (time.time()-start_time)

Starting genetic search
gen	nevals	avg     	min     	max     
0  	600   	0.657683	0.196681	0.797569
1  	359   	0.723069	0.209968	0.797569
2  	342   	0.754716	0.564417	0.807561
3  	369   	0.774149	0.635789	0.811469
4  	362   	0.782116	0.640882	0.811469
5  	360   	0.7872  	0.572518	0.811469
6  	373   	0.791681	0.589408	0.812213
7  	374   	0.795716	0.583869	0.81287 
8  	377   	0.800219	0.593392	0.812976
9  	344   	0.802568	0.614627	0.813007
10 	349   	0.804135	0.589636	0.813443
Best individual is: [1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
with fitness: (0.81344348720580273,)
The search took -1142 minutes.
68578.38678503036


In [ ]:
0.797194779659	0.794576261656	True	True	age	3,4,5,7,14,15
0.805916016012	0.818480415688	1.6243964317	True	True	pais_residencia','age','indrel','indrel_1mes','indext','segmento','month	1,2,5,6,10,11,16


## Submission

I have to create a submission function, I will reuse the one from the previous notebook.

In [34]:
def create_submission(filename, msg, 
                        verbose=False):
    """
    Implements all the steps of training and evaluating a naive bayes classifier
    Returns the score and the trained model
    
    train_month: int or list with the number of the month we want
        the data to be taken of for training 
    eval_month: int or list with the number of the month we want
        the data to be taken of for testing
    input_columns: a list with the name of the columns we are going to use
        in the task
    use_product: bool, if true adds the product columns of the month before
    use_change: bool, if true adds the change columns of the month before
    """
    test_month = 17
    #Train the model and get validation scores
    ret = naive_bayes_workflow(msg)
    scores = ret[0]
    bnb = ret[1]
    #Create a prediction
    msg['month'] = test_month
    predictions, output_data = create_prediction(bnb, msg)
    #Create the submission text
    if verbose: print ('Creating text...')
    text='ncodpers,added_products\n'
    for i, ncodpers in enumerate(dataset.eval_current[dataset.eval_current.fecha_dato == test_month].ncodpers):
        text += '%i,' % ncodpers
        for j in predictions[i]:
            text += '%s ' % dataset.product_columns[j]
        text += '\n'
    #Write to file
    if verbose: print ('Writing to file...')
    with gzip.open(dataset_root + 'submissions/%s.csv.gz' % filename, 'wb') as f:
        f.write(text.encode('utf-8'))
    
    return scores

In [36]:
#Create submission
start_time = time.time()
msg = {'train_month': [ 6,8,11,12,14,15],
       'eval_month': [5,16],
      'input_columns': ['ind_empleado','pais_residencia','age','indrel',
                        'indresi','indext','canal_entrada','renta'],
      'use_product': True,
      'use_change': True}
print (create_submission('NaiveBayes_8',msg))
print (time.time()-start_time)

[0.77461455927031386, 0.79387365791159514]
40.30816888809204


I get a LB score of 0.0256351

In [38]:
#Create submission
start_time = time.time()
msg = {'train_month': [ 3,4,11,15],
       'eval_month': [5,16],
      'input_columns': ['ind_empleado','pais_residencia','sexo',
                        'indrel_1mes','indfall','renta','segmento'],
      'use_product': True,
      'use_change': True}
print (create_submission('NaiveBayes_9',msg))
print (time.time()-start_time)

[0.78770011140463214, 0.79456803473556825]
36.07847595214844


I get a LB score of 0.0264266

In [39]:
#Create submission
start_time = time.time()
msg = {'train_month': [5],
       'eval_month': [5,16],
      'input_columns': ['pais_residencia','sexo','age','tiprel_1mes',
                        'canal_entrada','indfall','nomprov','renta'],
      'use_product': True,
      'use_change': True}
print (create_submission('NaiveBayes_10',msg))
print (time.time()-start_time)

[0.81097285715238709, 0.71654820129546593]
40.85058093070984


I get a LB score of 0.0256338.   
This is a clear example of overfitting, I’m getting a good score at month 5, but due to only train with that month the LB score is no better

In [ ]:
0.797194779659	0.794576261656	True	True	age	3,4,5,7,14,15
0.805916016012	0.818480415688	1.6243964317	True	True	pais_residencia','age','indrel','indrel_1mes','indext','segmento','month	1,2,5,6,10,11,16


In [40]:
#Create submission
start_time = time.time()
msg = {'train_month': [1,2,5,6,10,11,16],
       'eval_month': [5, 16],
      'input_columns': ['pais_residencia','age','indrel','indrel_1mes','indext','segmento','month'],
      'use_product': True,
      'use_change': True}
print (create_submission('NaiveBayes_11',msg))
print (time.time()-start_time)

[0.80591601601215246, 0.81848041568794927]
36.285911083221436


I get a LB score of , 87 in the classification( top 9%)  
That's very good for Naive Bayes

In [19]:
print (type(list(range(20000, 200001, 10000))))

<class 'list'>


In [18]:
used_months = np.random.randint(1,16,1)

In [19]:
print (used_months)

[12]
